Using regular logistic Regression (Highest Accuracy estimate(so far): 83%

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
d = pd.read_csv("student_lifestyle_dataset.csv")
Y = d["Stress_Level"]
dic = {"Moderate": 1, "High": 2, "Low": 0}
for i, element in np.ndenumerate(Y):
  Y[i,] = dic[element]
X = d.drop("Stress_Level", axis=1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)
w = [np.zeros(x_train.shape[1]) for i in range(3)]
b = np.array([0.0 for i in range(3)])

def softmax(z):
  e_z = np.exp(z - np.max(z))
  return e_z/sum(e_z)

def forwardPass(weights, x, biases):
  z = np.array([np.dot(w, x) + b for w, b in zip(weights, biases)])
  a = softmax(z)
  return z, a

def cross_entropy(y, y_pred):
  return -np.sum(y * np.log(y_pred + 1e-4))

def backprop(x, y, a):
  dz = a-y
  dw = [dz[i] * x for i in range(len(dz))]
  db = dz
  return dw, db

epochs = 2000
alpha = 0.01
for epoch in range(epochs):
  total_loss = 0
  for x, y in zip(x_train, y_train_one_hot):
    z, a = forwardPass(w, x, b)
    loss = cross_entropy(y, a)
    total_loss += loss
    dw, db = backprop(x, y, a)
    for i in range(3):
      w[i] -= alpha * dw[i]
      b[i] -= alpha * db[i]
  print("Loss at Epoch" + str(epoch +1)+ ": " + str(total_loss/len(x_train)))

In [ ]:
def predict(x, w, b):
  _, a = forwardPass(w, x, b)
  return np.argmax(a)

predictions = [predict(x, w, b) for x in x_test]
from sklearn.metrics import accuracy_score
y_test = np.array(y_test).astype(np.int64)
predictions = np.array(predictions).astype(np.int64)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy on test set:", accuracy)

Using Neural Network Model Highest Accuracy estimate (so far): 98%-97%

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
d = pd.read_csv("student_lifestyle_dataset.csv")
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
Y = d["Stress_Level"]
X = d.drop("Stress_Level", axis=1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

dic = {"Moderate": 1, "High": 2, "Low": 0}
print(Y.shape)
for i, element in np.ndenumerate(Y):
  Y[i,] = dic[element]
print(Y)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8)

y_train = y_train.astype(np.int64)
y_test = y_test.astype(np.int64)
model = keras.Sequential(
    [
        layers.Dense(activation="relu", units=125, kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dense(activation="relu", units=75, kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dense(activation="relu", units=15, kernel_regularizer=keras.regularizers.l2(0.01)),
        layers.Dense(activation="softmax", units=3)
    ]
)



optimizer = Adam(learning_rate=0.001)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=optimizer,
    metrics=['accuracy']
)

model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=200
)
